Pedersen Commitment
==================

## Pedersen commitment:

$Gen(1^\lambda) \rightarrow ck$

$$
g \leftarrow \mathbb{G}\\
h = g^x\\
ck:=(\mathbb{G}, p, g, h)
$$



$Com_{ck}(m) \rightarrow c$

$$
c:=(g^r, g^mh^r)
$$

In [1]:
from klefki.types.algebra.concrete import EllipticCurveCyclicSubgroupSecp256k1 as ECC
from klefki.types.algebra.concrete import EllipticCurveGroupSecp256k1 as Curve
from klefki.types.algebra.concrete import FiniteFieldCyclicSecp256k1 as CF
from klefki.types.algebra.concrete import FiniteFieldSecp256k1 as F
from klefki.types.algebra.utils import randfield
from klefki.utils import to_sha256int

G = ECC.G
H = Curve.lift_x(F(1234567))


### $\Sigma$-protocol

Consider a commitment $A$ opening to m to be part of the statement. The prover computes a random commitment $B = Com_{ck}(m; s)$ and sends it to the veriﬁer, which answer with a random challenge $x$. The prover then sends opening information $z$ to the veriﬁer, which checks the commitment $A^x B$ opens to m using randomness $z$.

$s \leftarrow \mathbb{Z}_p$
$B=Com_{ck}(m;s)$
$$
P \rightarrow V: B
$$

In [2]:
m = randfield(CF)
r = randfield(CF)

A = G ** m + H ** r

In [3]:
s = randfield(CF)


B = G ** s * H ** r

$x \leftarrow \mathbb{Z}_p$

$$
P \leftarrow V: x
$$

In [4]:
e = randfield(CF)


$z = me + s; x = re + r$

$$
P \rightarrow V: z, x
$$

In [5]:
z = m*e + s
x = r*e + r

Accept $\iff$ $B \in \mathbf{G}, z \in \mathbb{Z}_p$

$$
Com_{zk}(z;x) = A^eB
$$

In [6]:
G ** z * H ** x == A ** e * B

True

## Implementation

In [7]:
from klefki.zkp.pedersen import PedersonCommitment

In [8]:
#priv = randfield(CF)
secret = CF(73570390403507240989674623545632060650466613362119649500108200592951986722161)

r = randfield(CF)
e = randfield(CF)
P = PedersonCommitment(G, G@x, secret, r)

In [9]:
m = randfield(CF)
s = randfield(CF)


P.commit(m, s)

EllipticCurveGroupSecp256k1::(FiniteFieldSecp256k1::91958955717359388208542478944370361716254965936090085987507316561943720817853, FiniteFieldSecp256k1::35944189139569896164384495058484249049999039450507307656562740379781386245053)

In [10]:
P.challenge(e)

(FiniteFieldCyclicSecp256k1::84253794591609383410691939592927056153302076690673991993639785763012138005793,
 FiniteFieldCyclicSecp256k1::91592766882790668836854998962808066060615310035407545187287328919881095824502)

In [11]:
P.proof()

True

In [12]:
m1 = randfield(CF)


P.trapdoor(m1, x)

In [13]:
P.challenge(e)
P.proof()

True

## Ref:

* Eﬃcient Zero-Knowledge Proof Systems, Jonathan Bottle, ..., UCL
* Zero-Knowledge Proof and Cryptographic Commitment https://www.cs.purdue.edu/homes/ninghui/courses/555_Spring12/handouts/555_Spring12_topic23.pdf